# Request Resources

In [ ]:
%pip install --upgrade -q "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=expanse:shared
# %request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=AnvilCPU:shared
%wait_for_runtime hpc_cpu --live
%copy_data source=local:case.py target=hpc_cpu:case.py 
%switch_runtime hpc_cpu

# Run Job on Anvil

In [ ]:
! export MFC_Root="$HOME/MFC/MFC-Wilfong"; \
module load python openmpi/4.1.6 cmake/3.20.0 gcc/11.2.0; \
$MFC_Root/mfc.sh run case.py --case-optimization --no-gpu --mpi -b mpirun -N 1 -n 4 -j 4;

# Run Job on Expanse

In [ ]:
! export MFC_Root=/home/gridchem/apps/MFC/MFC-Wilfong; \
. $MFC_Root/mfc.sh load -c e -m c; \
module load cmake/3.21.4; \
$MFC_Root/mfc.sh run case.py --case-optimization --no-gpu --mpi -b mpirun -N 1 -n 4 -j 4 --no-build;

# Compress Database Files Transfer

In [ ]:
! tar -czf silo.tar.gz silo_hdf5/; \
echo "Tarball created"

# Transfer Files

In [ ]:
%copy_data source=hpc_cpu:silo.tar.gz target=local:silo.tar.gz